In [ ]:
import sys
import asyncio
import warnings
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass
from datetime import datetime, timedelta
import traceback
import sqlite3
import pandas as pd
import numpy as np
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
import pyqtgraph as pg
from pyqtgraph import PlotWidget, mkPen, mkBrush
import pyqtgraph.exporters
from concurrent.futures import ThreadPoolExecutor
import logging

try:
    from howtrader.trader.utility import BarGenerator, ArrayManager
    from howtrader.trader.object import BarData, TickData
    from howtrader.app.cta_strategy.template import CtaTemplate
    import talib
    HOWTRADER_AVAILABLE = True
except ImportError:
    HOWTRADER_AVAILABLE = False
    BarGenerator = None
    ArrayManager = None
    BarData = None
    TickData = None
    CtaTemplate = None

warnings.filterwarnings('ignore')
pg.setConfigOptions(antialias=True, useOpenGL=True)

In [ ]:
class TradingViewGrid:
    def __init__(self, plot_widget):
        self.plot_widget = plot_widget

    def apply_style(self):
        self.plot_widget.showGrid(x=True, y=True, alpha=0.3)
        self.plot_widget.setBackground('#1e222d')

        ax = self.plot_widget.getAxis('bottom')
        ax.setPen(pg.mkPen('#434651', width=1))
        ax.setTextPen(pg.mkPen('#d1d4dc'))

        ay = self.plot_widget.getAxis('left')
        ay.setPen(pg.mkPen('#434651', width=1))
        ay.setTextPen(pg.mkPen('#d1d4dc'))

        self.plot_widget.getViewBox().setMouseEnabled(x=True, y=True)
        self.plot_widget.setMouseEnabled(x=True, y=True)
        self.plot_widget.enableAutoRange(axis='y')

In [ ]:
class CandlestickItem(pg.GraphicsObject):
    def __init__(self, ohlc_data):
        pg.GraphicsObject.__init__(self)
        self.data = ohlc_data
        self.picture = QtGui.QPicture()
        self.generate_picture()

    def generate_picture(self):
        p = QtGui.QPainter(self.picture)
        p.setPen(pg.mkPen('w'))

        w = 0.8
        for i, (o, h, l, c) in enumerate(self.data):
            if c > o:
                p.setBrush(pg.mkBrush('#26a69a'))
                p.setPen(pg.mkPen('#26a69a'))
            else:
                p.setBrush(pg.mkBrush('#ef5350'))
                p.setPen(pg.mkPen('#ef5350'))

            p.drawLine(QtCore.QPointF(i, l), QtCore.QPointF(i, h))
            p.drawRect(QtCore.QRectF(i-w/2, min(o,c), w, abs(c-o)))

        p.end()

    def paint(self, p, *args):
        p.drawPicture(0, 0, self.picture)

    def boundingRect(self):
        return QtCore.QRectF(self.picture.boundingRect())

In [ ]:
class VolumeBarItem(pg.GraphicsObject):
    def __init__(self, volume_data, ohlc_data):
        pg.GraphicsObject.__init__(self)
        self.volume_data = volume_data
        self.ohlc_data = ohlc_data
        self.picture = QtGui.QPicture()
        self.generate_picture()

    def generate_picture(self):
        p = QtGui.QPainter(self.picture)

        w = 0.8
        max_vol = max(self.volume_data) if self.volume_data else 1

        for i, (vol, (o, h, l, c)) in enumerate(zip(self.volume_data, self.ohlc_data)):
            height = (vol / max_vol) * 100

            if c > o:
                p.setBrush(pg.mkBrush('#26a69a'))
                p.setPen(pg.mkPen('#26a69a'))
            else:
                p.setBrush(pg.mkBrush('#ef5350'))
                p.setPen(pg.mkPen('#ef5350'))

            p.drawRect(QtCore.QRectF(i-w/2, 0, w, height))

        p.end()

    def paint(self, p, *args):
        p.drawPicture(0, 0, self.picture)

    def boundingRect(self):
        return QtCore.QRectF(self.picture.boundingRect())

In [ ]:
class ImprovedTimeAxis(pg.AxisItem):
    def __init__(self, timestamps, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.timestamps = timestamps

    def tickStrings(self, values, scale, spacing):
        strings = []
        for v in values:
            if 0 <= int(v) < len(self.timestamps):
                ts = self.timestamps[int(v)]
                if isinstance(ts, str):
                    dt = datetime.fromisoformat(ts.replace('Z', '+00:00'))
                else:
                    dt = ts

                if spacing >= 86400:
                    strings.append(dt.strftime('%Y-%m-%d'))
                elif spacing >= 3600:
                    strings.append(dt.strftime('%m-%d %H:%M'))
                else:
                    strings.append(dt.strftime('%H:%M'))
            else:
                strings.append('')

In [ ]:
class IndicatorItem(pg.PlotDataItem):
    def __init__(self, indicator_type, data, color='white', **kwargs):
        super().__init__(**kwargs)
        self.indicator_type = indicator_type
        self.original_data = data

        if HOWTRADER_AVAILABLE and hasattr(self, 'apply_howtrader_smoothing'):
            processed_data = self.apply_howtrader_smoothing(data)
        else:
            processed_data = data

        self.setData(processed_data, pen=pg.mkPen(color, width=2))

    def apply_howtrader_smoothing(self, data):
        if len(data) < 5:
            return data
        try:
            smoothed = talib.SMA(np.array(data, dtype=float), timeperiod=3)
            return np.where(np.isnan(smoothed), data, smoothed)
        except:
            return data

In [ ]:
class SMCRectangleItem(pg.GraphicsObject):
    def __init__(self, x1, y1, x2, y2, color='yellow', alpha=0.3):
        pg.GraphicsObject.__init__(self)
        self.rect = QtCore.QRectF(x1, y1, x2-x1, y2-y1)
        self.color = color
        self.alpha = alpha

    def paint(self, p, *args):
        brush = pg.mkBrush(self.color)
        brush.setAlphaF(self.alpha)
        p.setBrush(brush)
        p.setPen(pg.mkPen(self.color, width=1))
        p.drawRect(self.rect)

    def boundingRect(self):

In [ ]:
class SMCLineItem(pg.GraphicsObject):
    def __init__(self, x1, y1, x2, y2, color='white', style='solid'):
        pg.GraphicsObject.__init__(self)
        self.line = QtCore.QLineF(x1, y1, x2, y2)
        self.color = color
        self.style = style

    def paint(self, p, *args):
        pen = pg.mkPen(self.color, width=2)
        if self.style == 'dashed':
            pen.setStyle(QtCore.Qt.DashLine)